In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import pickle
import glob
import json
import warnings
import pandas as pd
import numpy as np
import collections
from datetime import datetime
import getpass
from pathlib import Path

from helper_eval import evalAssays, evalClinvar, evalUKBB, evalDDD

aaPosCols = ["gene_name", "change_position_1based", "ref_aa", "alt_aa"]
dnaPosCols = ['chr', 'pos', 'non_flipped_ref', 'non_flipped_alt']

In [ ]:
datafolder = <PATH_TO_DATA>

precomputedScoreFilePaths = glob.glob( datafolder + "/pai_scores/*" )

precomputedScoreFilePaths

In [ ]:
scoreDF = None
for scoreFilePath in precomputedScoreFilePaths:
    print(scoreFilePath)
    
    df = pd.read_csv(scoreFilePath)
    df = df[ aaPosCols + [ col for col in df.columns if col.startswith("score") ] ].drop_duplicates(subset=aaPosCols)

    if scoreDF is None:
        scoreDF = df
    else:
        scoreDF = scoreDF.merge(df, on=aaPosCols)

In [9]:
scoreCols = [ col for col in scoreDF.columns if col.startswith("score") ]

In [10]:
scoreDF.head()

,gene_name,change_position_1based,ref_aa,alt_aa,score_PAI3D
0,ENST00000335137.4,15,E,G,0.489459
1,ENST00000335137.4,4,E,G,0.644101
2,ENST00000335137.4,276,N,D,0.816457
3,ENST00000335137.4,284,N,D,0.657667
4,ENST00000335137.4,288,K,E,0.657810


In [13]:
evalNameDict = {"dddAsdChd_2023": [datafolder + "/pai_trainAndEvalData/evaluation/dddAsdChd_2023.csv", evalDDD],
                "clinvar2023": [datafolder + "/pai_trainAndEvalData/evaluation/clinvar2023.csv", evalClinvar],
                "assays2023_02": [datafolder + "/pai_trainAndEvalData/evaluation/assays2023_02.csv", evalAssays],
                "bb2023_01": [datafolder + "/pai_trainAndEvalData/evaluation/bb2023_01.csv", evalUKBB]}

evalDFs = []

for evalName, (evalFilePath, evalFun) in evalNameDict.items():
    print(evalName)
    evalDF = evalFun(        evalFilePath,
                             scoreDF,
                             scoreCols)
    evalDFs.append(evalDF)

evalDF = pd.concat(evalDFs)

dddAsdChd_2023
clinvar2023
assays2023_02
bb2023_01


In [14]:
evalDF["statName"] = evalDF["dataset"] + "_" + evalDF["statName"]
evalDF.drop(columns=["dataset"], inplace=True)
evalDF = evalDF.set_index(['scoreName', "statName"])['stat_mean'].unstack(fill_value=0).reset_index()
evalDF.columns.name = "index"
evalDF

index,scoreName,DDD_asd_control_log10pval,DDD_chd_control_log10pval,DDD_ddd_control_log10pval,UKBB_corr,assay_corr,clinvar_auc
0,score_PAI3D,7.767343,3.99985,66.793698,0.283362,0.421079,0.927198
